In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Utilities
import re
import os
# Preprocessing
from html import unescape
from fuzzywuzzy import fuzz
# Data management
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("./data/book.txt", 
                   sep='\t', 
                   lineterminator='\n', 
                   header=None, 
                   names=["source", "isbn", "title", "authors"])

In [4]:
df.head()

,source,isbn,title,authors
0,eCampus.com,0201853949,"The art Of Computer Programming, Fascicle 3: G...",Not Available\r
1,Indoo.com,0201853949,"Art of Computer Programming, Volume 4, Fascicl...","Knuth, Donald E.\r"
2,textbookxdotcom,0201853949,"The 'art Of Computer Programming, Fascicle 3 G...",\r
3,A1Books,0201853949,"The Art of Computer Programming, Volume 4, Fas...","Knuth, Donald E.\r"
4,textbooksNow,0201853949,Art of Computer Programming,Knuth\r


In [5]:
df.describe()

,source,isbn,title,authors
count,33971,33971,33968,33971
unique,895,1265,11095,9627
top,A1Books,0321263588,Modern Database Management,\r
freq,2403,159,90,713


# Preprocessing

In [6]:
def clean(df, gs=False):
    if gs:
        df["authors"] = df["authors"].str.lower()
        df["authors"] = df["authors"].str.replace('[^\w\s]|\\r','')
        df.authors = df.authors.replace('\s+', ' ', regex=True)
        #df.authors = df.authors.replace(r'\b\w\b', ' ', regex=True)
    else:
        # Unescape HTML left over from scraping
        df.authors = df.authors.apply(unescape)
        df.title = df.title.apply(lambda x: unescape(str(x)))
        # Lowercase 
        df["authors"] = df["authors"].str.lower()
        df["title"] = df["title"].str.lower()
        df["source"] = df["source"].str.lower()
        # Removes parenthesis from author list
        df["authors"] = df["authors"].str.replace('\((.*?)\)','')
        # Replaces | in author list with spaces
        df["authors"] = df["authors"].str.replace('\|',' ')
        # Special characters removal (removes all special characters left)
        df["title"] = df["title"].str.replace('[^\w\s]|\\r','')
        df["authors"] = df["authors"].str.replace('[^\w\s]|\\r','')
        # Uniformize missing values
        df["authors"] = df["authors"].str.replace('not available','')
        df["authors"] = df["authors"].replace(r'^\s*$', np.nan, regex=True)
        # Remove doubled whitespaces
        df.authors = df.authors.replace('\s+', ' ', regex=True)
        df.title = df.title.replace('\s+', ' ', regex=True)
        # Remove single characters in authors names
        #df.authors = df.authors.replace(r'\b\w\b', ' ', regex=True)

In [7]:
clean(df)

In [8]:
df.describe()

,source,isbn,title,authors
count,33971,33971,33971,33227
unique,894,1265,7195,7234
top,a1books,0321263588,computer networking and the internet,knuth donald e
freq,2403,159,108,133


In [9]:
sum(df.duplicated())

7508

In [10]:
df = df.drop_duplicates()

# Data Quality Metrics

In [11]:
# Unique sources
sources = df.source.unique()

gs = pd.read_csv("./data/book_golden.txt", 
                   sep='\t', 
                   lineterminator='\n', 
                   header=None, 
                   names=["isbn", "authors"])
clean(gs, gs=True)
gs

,isbn,authors
0,9780073516677,oleary timothy j oleary linda i
1,9780072999389,yacht carol crosson susan
2,9780072922363,hutton david v
3,9780072843996,haag stephen perry james t sosinsky barrie est...
4,9780072466850,reddy j n
...,...,...
95,0122290631,eberly david h
96,0120887975,celko joe
97,0120455994,aiken peter allen m david
98,1555581994,cheek matthew


### Completeness
Attribute/Table completeness

NOTE: Closed World assumption

In [12]:
# Null values
null_count = df.isnull().sum()
print(null_count)

source       0
isbn         0
title        0
authors    602
dtype: int64


In [13]:
# Attribute completeness
rows = df.shape[0]
attributes = list(df)
attribute_compl = {}

for attribute in attributes:
    attribute_compl[attribute] = 1 - (null_count[attribute] / rows)

attribute_compl

{'source': 1.0, 'isbn': 1.0, 'title': 1.0, 'authors': 0.9772512564712995}

In [14]:
# Table completeness
table_compl = 1 - (null_count.sum() / (len(attributes)*rows))
print(table_compl)

0.9943128141178249


### Accuracy
For each source, matching of ISBN and authors list to Golden Standard

In [15]:
def compute_accuracy(source, gs):
    # Computes source accuracy by matching the golden standard rows to the source df, 
    # computing ISBN and Authors accuracy
        
    isbn_match_count = 0
    authors_match_count = 0
    authors_match_percentage = 0
    
    for gs_isbn, gs_authors in zip(gs.isbn, gs.authors):
        select = source[source.isbn == gs_isbn]
        select = select.dropna()
        if not select.empty:
            isbn_match_count += 1
            for authors in select.authors:
                tokens = authors.split()
                gs_tokens = gs_authors.split()
                match = fuzz.token_sort_ratio(tokens, gs_tokens)
                authors_match_percentage += match
                authors_match_count += 1
                
    acc = {
        "isbn": isbn_match_count/len(source.isbn),
        "authors": authors_match_percentage/authors_match_count
    }
    
    return acc

In [16]:
'''
source_accuracy = []

if not os.path.exists("./data/results/source_accuracy.csv"):
    for source in sources:
        selected_source = df[df.source == source]
        acc = compute_accuracy(df, gs)
        acc["source"] = source
        source_accuracy.append(acc)
    
    with open('./data/results/source_accuracy.csv', 'w') as f:
        f.write("{}; {}; {}".format("source", 
                                    "isbn coverage", 
                                    "authors accuracy"))
        for e in source_accuracy:
            f.write("{}; {}; {}".format(e["source"], 
                                        e["isbn"], 
                                        e["authors"]))
else:
    source_accuracy = pd.read_csv("./data/results/source_accuracy.csv", 
                   header=["source", "isbn coverage", "authors accuracy"])
'''

'\nsource_accuracy = []\n\nif not os.path.exists("./data/results/source_accuracy.csv"):\n    for source in sources:\n        selected_source = df[df.source == source]\n        acc = compute_accuracy(df, gs)\n        acc["source"] = source\n        source_accuracy.append(acc)\n    \n    with open(\'./data/results/source_accuracy.csv\', \'w\') as f:\n        f.write("{}; {}; {}".format("source", \n                                    "isbn coverage", \n                                    "authors accuracy"))\n        for e in source_accuracy:\n            f.write("{}; {}; {}".format(e["source"], \n                                        e["isbn"], \n                                        e["authors"]))\nelse:\n    source_accuracy = pd.read_csv("./data/results/source_accuracy.csv", \n                   header=["source", "isbn coverage", "authors accuracy"])\n'

### Coverage
How much data of the dataset each source covers

In [17]:
source_coverage = {}
unique_isbn = len(df.isbn.unique())

if not os.path.exists("./data/results/source_coverage.csv"):
    
    for source in sources:
        selected_source = df[df.source == source]
        source_coverage[source] = len(selected_source.isbn.unique()) / unique_isbn
    with open('./data/results/source_coverage.csv', 'w') as f:
            f.write("{}; {}\n".format("source","coverage"))
            for key, value in source_coverage.items():
                f.write("{}; {}\n".format(key, value))
                
else:
    source_coverage = pd.read_csv("data/results/source_coverage.csv", sep=";")

# Data Fusion

## TruthFinder implementation

In [18]:
from truthfinder import TruthFinder

In [19]:
df = df.dropna()

finder = TruthFinder(df, 
                     fact="authors", 
                     obj="isbn")

result = finder.compute()

D:\dev\truth_discovery\truthfinder.py:76: RuntimeWarning: divide by zero encountered in log
  sigma_f = sum(-np.log(1-t) for t in t_s)


TypeError: cannot do label indexing on <class 'pandas.core.indexes.numeric.Int64Index'> with these indexers [nan] of <class 'float'>

In [ ]:
# Saving results
result.to_csv("./data/results/truth.csv", index=False)
finder.df.to_csv("./data/results/complete.csv", index=False)